In [1]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [48]:
# Tool Creation

@tool
def multiply(a: int, b: int) -> int:
    """Given 2 numbers this tool returns their product"""
    return a*b

In [49]:
print(multiply.invoke({'a':3, 'b':4}))

12


In [50]:
multiply.name

'multiply'

In [51]:
multiply.description

'Given 2 numbers this tool returns their product'

In [52]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

In [53]:
from dotenv import load_dotenv
load_dotenv()

True

In [54]:
llm = HuggingFaceEndpoint(
    repo_id="openai/gpt-oss-20b",
    task="text-generation"
)

model = ChatHuggingFace(llm=llm)

In [55]:
llm_with_tools = model.bind_tools([multiply])

In [56]:
llm_with_tools.invoke('Hi how are you ?') 

AIMessage(content='Hello! I’m just a bunch of code, so I don’t have feelings, but I’m here and ready to help you. How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 134, 'total_tokens': 198}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--089b7806-ab5b-4ea6-96e4-95befd781f73-0', usage_metadata={'input_tokens': 134, 'output_tokens': 64, 'total_tokens': 198})

In [57]:
query = HumanMessage('can you multiply 3 with 10')

In [58]:
messages = [query]

In [59]:
messages

[HumanMessage(content='can you multiply 3 with 10', additional_kwargs={}, response_metadata={})]

In [60]:
result = llm_with_tools.invoke(messages)

In [61]:
messages.append(result)

In [62]:
messages

[HumanMessage(content='can you multiply 3 with 10', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"a":3,"b":10}', 'name': 'multiply', 'description': None}, 'id': 'call_0zstEbl2TCm13Q99PaUaKaXl', 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 137, 'total_tokens': 171}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--c3e3b4e8-9303-4c09-889c-916407007b66-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 10}, 'id': 'call_0zstEbl2TCm13Q99PaUaKaXl', 'type': 'tool_call'}], usage_metadata={'input_tokens': 137, 'output_tokens': 34, 'total_tokens': 171})]

In [63]:
tool_result = multiply.invoke(result.tool_calls[0])

In [64]:
messages.append(tool_result)

In [65]:
messages

[HumanMessage(content='can you multiply 3 with 10', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"a":3,"b":10}', 'name': 'multiply', 'description': None}, 'id': 'call_0zstEbl2TCm13Q99PaUaKaXl', 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 137, 'total_tokens': 171}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--c3e3b4e8-9303-4c09-889c-916407007b66-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 10}, 'id': 'call_0zstEbl2TCm13Q99PaUaKaXl', 'type': 'tool_call'}], usage_metadata={'input_tokens': 137, 'output_tokens': 34, 'total_tokens': 171}),
 ToolMessage(content='30', name='multiply', tool_call_id='call_0zstEbl2TCm13Q99PaUaKaXl')]

In [66]:
llm_with_tools.invoke(messages).content

'30'